## PDS Group 7

In [1]:
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)

In [2]:
# Load the processed ARCOs data and display its info
arcos = pd.read_parquet("arcos_processed.parquet")
arcos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162771 entries, 0 to 162770
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   BUYER_STATE   162771 non-null  object 
 1   BUYER_COUNTY  162771 non-null  object 
 2   YEAR          162771 non-null  int32  
 3   TOTAL_MME     162771 non-null  float64
dtypes: float64(1), int32(1), object(2)
memory usage: 4.3+ MB


In [3]:
# check for missing values in each column
arcos.isna().sum()

BUYER_STATE     0
BUYER_COUNTY    0
YEAR            0
TOTAL_MME       0
dtype: int64

In [4]:
arcos.sample(15)

,BUYER_STATE,BUYER_COUNTY,YEAR,TOTAL_MME
30880,FL,BROWARD,2008,2.955537e+07
143055,NC,WATAUGA,2009,2.599583e+03
21752,NC,ROBESON,2009,8.646196e+01
144068,WA,DOUGLAS,2014,1.235365e+03
25664,SC,MCCORMICK,2009,5.351302e+01
14367,NC,CRAVEN,2016,2.489738e+04
66873,GA,LAURENS,2014,2.020434e+03
78455,GA,CLAY,2009,5.452576e+01
96200,GA,SCREVEN,2010,9.473574e+01
19722,WA,ADAMS,2018,6.414948e+02


In [5]:
arcos["merge_on"] = (
    arcos["BUYER_COUNTY"].str.upper() + " COUNTY, " + arcos["BUYER_STATE"]
)

In [6]:
arcos["merge_on"].sample(10)

139557      CATAWBA COUNTY, NC
120873       DURHAM COUNTY, NC
124505    HENDERSON COUNTY, NC
1304          UPSON COUNTY, GA
27539     CLACKAMAS COUNTY, OR
17613      ATKINSON COUNTY, GA
136140        ROWAN COUNTY, NC
76156        JASPER COUNTY, GA
9184            LEE COUNTY, NC
74789        JASPER COUNTY, GA
Name: merge_on, dtype: object

In [7]:
pop = pd.read_csv("merged_mortality_population.csv")
pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61559 entries, 0 to 61558
Data columns (total 13 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   County                           61559 non-null  object
 1   County Code                      61559 non-null  int64 
 2   Year                             61559 non-null  int64 
 3   Drug/Alcohol Induced Cause       61559 non-null  object
 4   Drug/Alcohol Induced Cause Code  61559 non-null  object
 5   Deaths                           61559 non-null  int64 
 6   STATE                            61559 non-null  int64 
 7   COUNTY                           61559 non-null  int64 
 8   STNAME                           61559 non-null  object
 9   CTYNAME                          61559 non-null  object
 10  year                             61559 non-null  int64 
 11  population                       61559 non-null  int64 
 12  fips                            

In [8]:
pop.sample(10)

,County,County Code,Year,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,STATE,COUNTY,STNAME,CTYNAME,year,population,fips
8459,"Imperial County, CA",6025,2005,Drug poisonings (overdose) Unintentional (X40-...,D1,13,6,25,California,Imperial County,2005,156113,6025
50638,"Harrison County, OH",39067,2013,All other non-drug and non-alcohol causes,O9,168,39,67,Ohio,Harrison County,2013,15602,39067
48128,"Tolland County, CT",9013,2013,All other non-drug and non-alcohol causes,O9,972,9,13,Connecticut,Tolland County,2013,151778,9013
1128,"Decatur County, IA",19053,2003,All other non-drug and non-alcohol causes,O9,108,19,53,Iowa,Decatur County,2003,8575,19053
29801,"Mobile County, AL",1097,2010,All other alcohol-induced causes,A9,42,1,97,Alabama,Mobile County,2010,413277,1097
5084,"Shelby County, IL",17173,2004,All other non-drug and non-alcohol causes,O9,238,17,173,Illinois,Shelby County,2004,22484,17173
38531,"Park County, WY",56029,2010,All other non-drug and non-alcohol causes,O9,245,56,29,Wyoming,Park County,2010,28271,56029
61091,"Bath County, VA",51017,2015,All other non-drug and non-alcohol causes,O9,66,51,17,Virginia,Bath County,2015,4460,51017
59028,"Pontotoc County, MS",28115,2015,All other non-drug and non-alcohol causes,O9,281,28,115,Mississippi,Pontotoc County,2015,30857,28115
55694,"Berkeley County, SC",45015,2014,All other non-drug and non-alcohol causes,O9,1315,45,15,South Carolina,Berkeley County,2014,198295,45015


In [9]:
merged = arcos.merge(
    pop, how="inner", right_on=["County", "year"], left_on=["merge_on", "YEAR"]
)

In [10]:
merged

,BUYER_STATE,BUYER_COUNTY,YEAR,TOTAL_MME,merge_on,County,County Code,Year,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,STATE,COUNTY,STNAME,CTYNAME,year,population,fips
